In [27]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [49]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('mmdetection/configs/yolox/yolox_s.py')

root='../dataset/'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.pipeline[1]['img_scale'] = (640,640) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [50]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [51]:
epoch = 'best_bbox_mAP_50_epoch_50'

In [80]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector

In [53]:
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/inseo_test/yolox_s(no mix up)/best_bbox_mAP_50_epoch_50.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, ema_backbone_stage1_1_main_conv_bn_running_var, ema_backbone_stage1_1_main_conv_bn_num_batches_tracked, ema_backbone_stage1_1_short_conv_conv_weight, ema_backbone_stage1_1_shor

In [25]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 27.8 task/s, elapsed: 175s, ETA:     0s

In [26]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


,PredictionString,image_id
0,0 0.17172451 60.22257 425.0174 157.7675 616.49...,test/0000.jpg
1,0 0.8154744 764.34595 659.1841 902.8854 981.43...,test/0001.jpg
2,0 0.39542922 868.24316 251.29037 913.5873 325....,test/0002.jpg
3,0 0.30983406 151.6824 256.57733 908.03845 807....,test/0003.jpg
4,0 0.89017373 415.41544 408.85974 657.4796 574....,test/0004.jpg


# for Yolo X (EMA model)

In [255]:
import torch

In [256]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
model.eval()

YOLOX(
  (backbone): CSPDarknet(
    (stem): Focus(
      (conv): ConvModule(
        (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
    )
    (stage1): Sequential(
      (0): ConvModule(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
      (1): CSPLayer(
        (main_conv): ConvModule(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (activate): Swish()
        )
        (short_conv): ConvModule(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, 

In [257]:
checkpoint = torch.load(checkpoint_path)['state_dict']

In [264]:
new_checkpoint = {}
for key,val in checkpoint.items():
    print(key,val)
    break

ema_backbone_stem_conv_conv_weight tensor([[[[ 4.5660e-02,  1.2302e-02, -4.4571e-02],
          [ 2.2522e-01,  1.2401e-01, -2.1114e-01],
          [ 1.4896e-01,  8.3583e-02, -1.6309e-01]],

         [[ 2.2983e-02,  1.1856e-02, -2.9822e-02],
          [ 2.4942e-01,  1.2509e-01, -2.5352e-01],
          [ 1.7355e-01,  8.7825e-02, -1.9363e-01]],

         [[ 4.2650e-02,  2.0060e-02, -3.2249e-02],
          [ 2.5750e-01,  1.1823e-01, -2.3184e-01],
          [ 1.8271e-01,  8.2504e-02, -1.8220e-01]],

         ...,

         [[ 1.1812e-01, -3.4557e-02, -1.2388e-01],
          [ 2.5564e-01, -3.9925e-02, -3.1626e-01],
          [ 6.6711e-02, -4.0882e-02, -6.7698e-02]],

         [[ 1.1522e-01, -4.8277e-02, -1.3514e-01],
          [ 2.8177e-01, -6.5522e-02, -3.6165e-01],
          [ 7.8421e-02, -4.6518e-02, -9.4212e-02]],

         [[ 1.1861e-01, -5.2074e-02, -1.3830e-01],
          [ 2.7182e-01, -5.9640e-02, -3.4638e-01],
          [ 8.5204e-02, -3.9904e-02, -9.0349e-02]]],


        [[[-5.3959

In [258]:
#for name, value in model.modules():
new_checkpoint = {}
for name,_ in model.named_parameters():
    new_checkpoint[name] = checkpoint["ema_"+name.replace('.','_')]
    #print(name)

In [259]:
model.load_state_dict(new_checkpoint,strict=False)

_IncompatibleKeys(missing_keys=['backbone.stem.conv.bn.running_mean', 'backbone.stem.conv.bn.running_var', 'backbone.stage1.0.bn.running_mean', 'backbone.stage1.0.bn.running_var', 'backbone.stage1.1.main_conv.bn.running_mean', 'backbone.stage1.1.main_conv.bn.running_var', 'backbone.stage1.1.short_conv.bn.running_mean', 'backbone.stage1.1.short_conv.bn.running_var', 'backbone.stage1.1.final_conv.bn.running_mean', 'backbone.stage1.1.final_conv.bn.running_var', 'backbone.stage1.1.blocks.0.conv1.bn.running_mean', 'backbone.stage1.1.blocks.0.conv1.bn.running_var', 'backbone.stage1.1.blocks.0.conv2.bn.running_mean', 'backbone.stage1.1.blocks.0.conv2.bn.running_var', 'backbone.stage2.0.bn.running_mean', 'backbone.stage2.0.bn.running_var', 'backbone.stage2.1.main_conv.bn.running_mean', 'backbone.stage2.1.main_conv.bn.running_var', 'backbone.stage2.1.short_conv.bn.running_mean', 'backbone.stage2.1.short_conv.bn.running_var', 'backbone.stage2.1.final_conv.bn.running_mean', 'backbone.stage2.1.fin

In [260]:
# checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [261]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 36.2 task/s, elapsed: 134s, ETA:     0s

In [262]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,,test/0000.jpg
1,,test/0001.jpg
2,,test/0002.jpg
3,,test/0003.jpg
4,,test/0004.jpg
